<a href="https://colab.research.google.com/github/LeeHonggii/hair_loss_problem_model./blob/main/ourmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/nvidia/our_dataset.zip

Archive:  /content/drive/MyDrive/nvidia/our_dataset.zip
   creating: our_dataset/
  inflating: our_dataset/.DS_Store   
  inflating: __MACOSX/our_dataset/._.DS_Store  
   creating: our_dataset/Ture/
  inflating: __MACOSX/our_dataset/._Ture  
   creating: our_dataset/False/
  inflating: __MACOSX/our_dataset/._False  
  inflating: our_dataset/Ture/True-103.jpeg  
  inflating: __MACOSX/our_dataset/Ture/._True-103.jpeg  
  inflating: our_dataset/Ture/True-046.jpeg  
  inflating: __MACOSX/our_dataset/Ture/._True-046.jpeg  
  inflating: our_dataset/Ture/True-154.jpeg  
  inflating: __MACOSX/our_dataset/Ture/._True-154.jpeg  
  inflating: our_dataset/Ture/True-011.jpeg  
  inflating: __MACOSX/our_dataset/Ture/._True-011.jpeg  
  inflating: our_dataset/Ture/True-142.jpeg  
  inflating: __MACOSX/our_dataset/Ture/._True-142.jpeg  
  inflating: our_dataset/Ture/True-007.jpeg  
  inflating: __MACOSX/our_dataset/Ture/._True-007.jpeg  
  inflating: our_dataset/Ture/True-115.jpeg  
  inflating: __MAC

In [3]:
from glob import glob
mother_path = '/content/our_dataset'

glob(mother_path+'/*/*')

['/content/our_dataset/False/False-27.jpeg',
 '/content/our_dataset/False/False-37.jpeg',
 '/content/our_dataset/False/False-33.jpeg',
 '/content/our_dataset/False/False-44.jpeg',
 '/content/our_dataset/False/False-23.jpeg',
 '/content/our_dataset/False/False-40.jpeg',
 '/content/our_dataset/False/False-12.jpeg',
 '/content/our_dataset/False/False-42.jpeg',
 '/content/our_dataset/False/False-31.jpeg',
 '/content/our_dataset/False/False-34.jpeg',
 '/content/our_dataset/False/False-19.jpeg',
 '/content/our_dataset/False/False-39.jpeg',
 '/content/our_dataset/False/False-14.jpeg',
 '/content/our_dataset/False/False-36.jpeg',
 '/content/our_dataset/False/False-06.jpeg',
 '/content/our_dataset/False/False-25.jpeg',
 '/content/our_dataset/False/False-18.jpeg',
 '/content/our_dataset/False/False-11.jpeg',
 '/content/our_dataset/False/False-32.jpeg',
 '/content/our_dataset/False/False-46.jpeg',
 '/content/our_dataset/False/False-51.jpeg',
 '/content/our_dataset/False/False-17.jpeg',
 '/content

In [15]:
import os
from PIL import Image
import numpy as np
folder_list = ['Ture','False']
train = []
label = []
for idx, folder_name in enumerate(folder_list):
    folder = os.path.join(mother_path, folder_name)
    all_files_path = glob(folder+'/*')
    for k,file_path in enumerate(all_files_path):
      img = Image.open(file_path)
      img = img.resize((128,128))
      img = img.convert('RGB')
      np_img = np.array(img)
      train.append(np_img)
      label.append(idx)
      if k % 10 == 0:
            print(f'{k} / {len(all_files_path)}')
np_train = np.array(train)
np_label = np.array(label)

0 / 167
10 / 167
20 / 167
30 / 167
40 / 167
50 / 167
60 / 167
70 / 167
80 / 167
90 / 167
100 / 167
110 / 167
120 / 167
130 / 167
140 / 167
150 / 167
160 / 167
0 / 54
10 / 54
20 / 54
30 / 54
40 / 54
50 / 54


In [5]:
print(np.bincount(np_label))
print(np_train.shape)

[167  54]
(221, 128, 128, 3)


In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(np_train,np_label,test_size=0.2)

In [7]:
#xy = (x_train,x_test,y_train,y_test)
#np

In [8]:
x_train = x_train/255.
x_test = x_test/255.

In [9]:
import tensorflow as tf
from tensorflow.keras import layers,models

input_layer = layers.Input(shape=(128,128,3), name='start layer')
tower_1 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_1' )(input_layer)
tower_2 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_2_1')(input_layer)
tower_2 = layers.Conv2D(64 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_2')(tower_2)
tower_3 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_1')(input_layer)
tower_3 = layers.Conv2D(64 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_2')(tower_3)
tower_4 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_4_1')(input_layer)
tower_4 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_4_2')(tower_4)
#
concat_layer = layers.concatenate([tower_1,tower_2,tower_3,tower_4],axis=3)
concat_layer = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer)
##
tower_2_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_2_1_1' )(concat_layer)
tower_2_2_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_2_2_1')(concat_layer)
tower_2_2_2 = layers.Conv2D(120 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_2_2')(tower_2_2_1)
tower_2_3_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_3_1')(concat_layer)
tower_2_3_2 = layers.Conv2D(120 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_3_2')(tower_2_3_1)
tower_2_4_1 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_2_4_1')(concat_layer)
tower_2_4_2 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_4_2')(tower_2_4_1)
concat_layer_2 = layers.concatenate([tower_2_1,tower_2_2_2,tower_2_3_2,tower_2_4_2],
                                  axis=3)
concat_layer_2 = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer_2)
#
last_avg_pool = layers.AveragePooling2D(pool_size=(31,31) , strides=(1,1), padding='valid')(concat_layer_2)
flat_layer = layers.Flatten()(last_avg_pool)
output_layer = layers.Dense(1 ,activation='sigmoid')(flat_layer)
model = models.Model(input_layer,output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start layer (InputLayer)    [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 tower_2_1 (Conv2D)          (None, 128, 128, 64)         256       ['start layer[0][0]']         
                                                                                                  
 tower_3_1 (Conv2D)          (None, 128, 128, 64)         256       ['start layer[0][0]']         
                                                                                                  
 tower_4_1 (MaxPooling2D)    (None, 128, 128, 3)          0         ['start layer[0][0]']         
                                                                                              

In [10]:
!mkdir tensorboard_log

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])
tb = tf.keras.callbacks.TensorBoard(log_dir='/content/tensorboard_log',
                                    histogram_freq=1)
model.fit(x_train,y_train,
          validation_data=(x_test,y_test),
          epochs=10,callbacks=[tb])


Epoch 1/10
6/6 [==============================] - 26s 2s/step - loss: 0.5811 - accuracy: 0.7727 - val_loss: 0.6119 - val_accuracy: 0.6889
Epoch 2/10
6/6 [==============================] - 2s 272ms/step - loss: 0.5370 - accuracy: 0.7727 - val_loss: 0.5941 - val_accuracy: 0.6889
Epoch 3/10
6/6 [==============================] - 2s 283ms/step - loss: 0.5191 - accuracy: 0.7727 - val_loss: 0.6393 - val_accuracy: 0.6889
Epoch 4/10
6/6 [==============================] - 2s 280ms/step - loss: 0.5364 - accuracy: 0.7727 - val_loss: 0.6184 - val_accuracy: 0.6889
Epoch 5/10
6/6 [==============================] - 2s 280ms/step - loss: 0.5260 - accuracy: 0.7727 - val_loss: 0.5936 - val_accuracy: 0.6889
Epoch 6/10
6/6 [==============================] - 2s 283ms/step - loss: 0.5248 - accuracy: 0.7727 - val_loss: 0.6159 - val_accuracy: 0.6889
Epoch 7/10
6/6 [==============================] - 2s 306ms/step - loss: 0.5217 - accuracy: 0.7727 - val_loss: 0.6050 - val_accuracy: 0.6889
Epoch 8/10
6/6 [======

In [17]:
from google.colab import files
f = files.upload()

Saving True-003.jpeg to True-003.jpeg


In [18]:
test_file_name = list(f.keys())[0]
img = Image.open(file_path)
img = img.resize((128,128))
img = img.convert('RGB')
data = np.asarray(img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor/255.
test_image_tensor.shape

(1, 128, 128, 3)

In [19]:
result = model.predict(test_image_tensor)
if result[0][0] > 0.5:
    print(f'{folder_list[1]} 일 확률이 {result[0][0] * 100} % 입니다')
else:
    print(f'{folder_list[0]} 일 확률이 {(1 - result[0][0])* 100} % 입니다')


1/1 [==============================] - 0s 19ms/step
Ture 일 확률이 69.503054022789 % 입니다
